In [1]:
import json
from collections import defaultdict
import math

# Global constants
MAX_RECURSION_DEPTH = 6  # Default max recursion depth

def load_data():
    """Load the Satisfactory data from the JSON file."""
    with open("raw_data/data.json", 'r', encoding='utf-8') as file:
        return json.load(file)


def display_recipes(data, recipes):
    """Display the recipes in a readable format."""
    if not recipes:
        print("No recipes found.")
        return

    print("Recipes:")
    print("=" * 80)
    for recipe_id in recipes:
        recipe_data = data['recipes'].get(recipe_id, {})
        recipe_name = recipe_data.get('name', recipe_id)
        print(f"Recipe ID: {recipe_id} - {recipe_name}")
        print(f"  Machine: {recipe_data.get('machine', 'Unknown')}")
        print(f"  Time: {recipe_data.get('time', 1.0)} seconds")
        print(f"  Power Usage: {recipe_data.get('power_use', 0)} MW")
        print("  Products:")
        for product in recipe_data.get('products', []):
            item = product.get('item')
            amount = product.get('amount', 1)
            print(f"    - {item} x{amount}")
        print("  Ingredients:")
        for ingredient in recipe_data.get('ingredients', []):
            item = ingredient.get('item')
            amount = ingredient.get('amount', 1)
            print(f"    - {item} x{amount}")
        print()

def is_valid_item(data, item_id):
    """Check if an item ID is valid."""
    if item_id not in data.get('items', {}) and item_id not in data.get('resources', {}):
        #print(f"Warning: Item '{item_id}' does not exist in the data!")
        return False
    return True

def is_resource(data, item_id):
    """Check if an item is a resource."""
    if not is_valid_item(data, item_id):
        return False
    return item_id in data.get('resources', {})

def is_crafted(data, item_id):
    """Check if an item is crafted by a recipe."""
    if not is_valid_item(data, item_id):
        return False
    if is_resource(data, item_id):
        return False
    for recipe_id, recipe_data in data['recipes'].items():
        if not recipe_data.get('products'):
            continue
        for product in recipe_data.get('products', []):
            if not isinstance(product, dict):
                continue
            product_item = product.get('item')
            if product_item == item_id:
                return True
    return False


def get_item_recipes(data,item_id):
    """Get all recipes that produce the given item."""
    recipes = []
    # Find recipes that produce this item
    for recipe_id, recipe_data in data['recipes'].items():
        if not recipe_data.get('products'):
            continue
        if False or "unpackage" in recipe_id.lower():
            #print(f"Skipping unpackage recipe: {recipe_id}")
            continue
        for product in recipe_data.get('products', []):
            if not isinstance(product, dict):
                continue

            product_item = product.get('item')
            if product_item == item_id:
                recipe_data["recipe_id"] = recipe_id
                recipes.append(recipe_data)
    return recipes

def display_recipe(recipe):
    print(f"{recipe.get('name', 'Unknown')}")
    #print(f"  Machine: {recipe.get('machine', 'Unknown')}, Time: {recipe.get('time', 1.0)} seconds")
    print("  Products:")
    for product in recipe.get('products', []):
        item = product.get('item')
        amount = product.get('amount', 1)
        print(f"    - {item} x{amount}")
    print("  Ingredients:")
    for ingredient in recipe.get('ingredients', []):
        item = ingredient.get('item')
        amount = ingredient.get('amount', 1)
        print(f"    - {item} x{amount}")



def add_prod_rate(data):
    """ Add production rate per minute for each recipe """
    for recipe_id, recipe_data in data['recipes'].items():
        products = recipe_data.get('products', [])
        if not products:
            continue
        # Add the rate for each products
        for product in products:
            if not isinstance(product, dict):
                continue
            item = product.get('item')
            amount = product.get('amount', 1)
            # Calculate the production rate per minute
            rate = round(60 / recipe_data.get('time', 1.0) * amount, 3)
            product['rate'] = rate
        # Add the production rate for the ingredients
        ingredients = recipe_data.get('ingredients', [])
        for ingredient in ingredients:
            if not isinstance(ingredient, dict):
                continue
            item = ingredient.get('item')
            amount = ingredient.get('amount', 1)
            # Calculate the consumption rate per minute
            rate = round((60 / recipe_data.get('time', 1.0)) * amount, 3)
            ingredient['rate'] = rate

def add_resource_tag(data):
    """ Add a 'resource' tag to each ingredients of each recipes """
    for recipe_id, recipe_data in data['recipes'].items():
        ingredients = recipe_data.get('ingredients', [])
        if not ingredients:
            continue
        for ingredient in ingredients:
            if not isinstance(ingredient, dict):
                continue
            item = ingredient.get('item')
            if is_resource(data, item):
                ingredient['resource'] = True
            else:
                ingredient['resource'] = False

def detect_circular_recipes(recipe, lookup_recipe_id):
    """
    Detect if a recipe_id is already use in the ingredients of the recipes, for the whole chain
    Args:
        data (_type_): _description_
        recipe_id (_type_): _description_
    """
    for ingredient in recipe.get('ingredients', []):
        if 'recipes' in ingredient:
            for ir in ingredient['recipes']:
                if ir.get('recipe_id') == lookup_recipe_id:
                    return True
                # Check recursively in the sub-recipes
                if detect_circular_recipes(ir['recipe'], lookup_recipe_id):
                    return True
    else:
        return False


def add_ingredients_recipes(data):
    """
    Add a list of recipe_id for each ingredients of each recipes
    Also add a factor to match the rate of the ingredient
    Only ingredient that are not resource are processed
    """
    for recipe_id, recipe_data in data['recipes'].items():
        ingredients = recipe_data.get('ingredients', [])
        if not ingredients:
            continue
        for ingredient in ingredients:
            if not isinstance(ingredient, dict):
                continue
            if ingredient.get('resource', False):
                # Skip resources, they don't have recipes
                continue
            item = ingredient.get('item')
            # Get the recipes that produce this item
            ingredient_recipes = get_item_recipes(data, item)
            if not ingredient_recipes:
                print(f"Warning: No recipes found for ingredient '{item}' in recipe '{recipe_id}'")
                continue
            ingredient['recipes'] = []
            for ir in ingredient_recipes:
                # Get the desired ingredient from the products of the recipe, anmd calculate the factor
                if ir.get('recipe_id') == recipe_id:
                    # Skip the current recipe, as it is already processed
                    #print(f"Skipping current recipe '{recipe_id}' for ingredient '{item}' to avoid circular reference")
                    continue
                for product in ir.get('products', []):
                    if not isinstance(product, dict):
                        continue
                    if product.get('item') == item:
                        if item in [ir_ingredient['item'] for ir_ingredient in ir.get('ingredients', [])]:
                            print(f"Warning: Ingredient '{item}' in recipe '{recipe_id}' is also an ingredient in recipe '{ir['recipe_id']}'")
                            continue
                        factor = ingredient.get('rate', 1) / product.get('rate', 1)
                        break
                else:
                    # If no matching product found, set factor to 1
                    #print(f"Warning: No matching product found for ingredient '{item}' in recipe '{recipe_id}'")
                    factor = 1
                ingredient['recipes'].append({
                    'recipe_id': ir['recipe_id'],
                    'factor': factor,
                    'ingredients_id': [ir_ingredient['item'] for ir_ingredient in ir.get('ingredients', [])],
                })




In [2]:
import json
data = load_data()
add_prod_rate(data)
add_resource_tag(data)
add_ingredients_recipes(data)


with open("raw_data/enhanced_recipes.json", 'w', encoding='utf-8') as file:
    json.dump(data['recipes'], file, indent=4, ensure_ascii=False)

